<a href="https://colab.research.google.com/github/Andy-Lewis-Sapner/AIT-G21-SemB24-Ex1/blob/main/Exercises/Exercise4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [496]:
# Mount google drive to get access to the data
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [497]:
# Imports needed for this exercise
import matplotlib.pyplot as plt
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML

In [498]:
# Helper class to keep track of index and data in display
class data_holder:
  def set_index(self, index):
    self.index = index

  def get_index(self):
    return self.index

  def set_data(self, data, key):
    # Sort data based on key
    self.data = dict(sorted(data.items()))
    key_text.value = ""
    value_text.value = ""
    key_text.description = "<font color='orange'>" + key + ": </font>"

  def get_data(self):
    return self.data

  def set_data_in_display(self):
    key_text.value = list(self.data.keys())[self.index]
    value_text.value = str(self.data[key_text.value])

In [499]:
# Helper function to create bar graph from data holder that is displayed
def key_bar_graph(df, key):
  if not key in df.keys():
    return

  output = widgets.Output(
    layout=widgets.Layout(
      display='flex',
      flex_flow='column',
      align_items='center',
      border="4px solid black"
    )
  )

  displayed_data = {}
  for _, row in df.iterrows():
    data_str = row.get(key)
    if data_str is None:
      continue

    # Trim data for the "Time" column
    if key == "Time":
      data_str = data_str[:4]

    # Count number of occurences of each unique value in the column
    if data_str in displayed_data:
      displayed_data[data_str] += 1
    else:
      displayed_data[data_str] = 1

  # Set the needed data in the data holder and the index to its initial value (-1)
  data_holder.set_data(displayed_data, key)
  data_holder.set_index(-1)

  # Filter out values that are less than 10% of the average
  if key == "Document":
    values_average = sum(displayed_data.values()) / len(displayed_data.values())
    displayed_data = {key: value for key, value in displayed_data.items() if value > 0.1 * values_average}

  # Create the bar graph and display it via output
  with output:
    plt.bar(displayed_data.keys(), displayed_data.values(), align='center')
    plt.title(key)
    plt.rc('font', size=8)
    plt.show()

  return output

In [500]:
# Event handler for dropdown, to update the bar graph and the data prev-next displayer when a new value is selected
def dropdown_eventhandler(change):
   if change['type'] == 'change' and change['name'] == 'value':
        new_value = change['new']
        output = key_bar_graph(df, new_value)
        children = list(vbox.children)
        if len(children) > 1:
          children = children[:1]
        output_hbox = widgets.HBox(children=[output, data_vbox], layout=widgets.Layout(margin="20px auto"))
        children.append(output_hbox)
        vbox.children = children

In [501]:
# Event handler for prev button
def prev_button_clicked(b):
  index = data_holder.get_index()
  if index > 0:
    data_holder.set_index(index - 1)
    data_holder.set_data_in_display()

In [502]:
# Event handler for next button
def next_button_clicked(b):
  index = data_holder.get_index()
  if index < len(data_holder.get_data().keys()) - 1:
    data_holder.set_index(index + 1)
    data_holder.set_data_in_display()

In [503]:
# Load data from the json file in google drive
path = '/content/drive/My Drive/Colab Notebooks/IntroToCloudComputing/data.json'
df = pd.read_json(path)

# Initialize the data holder
data_holder = data_holder()

In [504]:
# Create the widgets for the data prev-next displayer
key_text = widgets.Text(value=None, description="<font color='orange'>Key: </font>", layout=widgets.Layout(width="95%"), disabled=True)
value_text = widgets.Text(value=None, description="<font color='orange'>Counter: </font>", layout=widgets.Layout(width="95%"), disabled=True)

prev_button = widgets.Button(
  description = "Previous",
  layout = widgets.Layout(width="95%", cursor="pointer")
)
prev_button.on_click(prev_button_clicked)
prev_button.style.button_color = "#ff6666"

next_button = widgets.Button(
  description = "Next",
  layout = widgets.Layout(width="95%", cursor="pointer")
)
next_button.on_click(next_button_clicked)
next_button.style.button_color = "#ff6666"

buttons_hbox = widgets.HBox(
    children=[prev_button, next_button],
    layout=widgets.Layout(width="95%", display='flex', flex_flow='row', justify_content='space-between')
)

data_vbox = widgets.VBox(
    children=[key_text, value_text, buttons_hbox],
    layout=widgets.Layout(
      display='flex',
      flex_flow='column',
      align_items='center',
      margin="12px")
)

In [505]:
# Create the main display with the dropdown selector
dropdown = widgets.Dropdown(
  options=["Time", "Document", "User"],
  value=None,
  description='<font color=\'orange\'>Graph: </font>'
)
dropdown.observe(dropdown_eventhandler)

vbox = widgets.VBox(
    children=[dropdown],
    layout=widgets.Layout(
      display='flex',
      flex_flow='column',
      align_items='center',
      margin="5px auto")
)
display(vbox)

custom_css = """
<style>
    .widget-dropdown select {
        color: orange;
    }
</style>
"""
display(HTML(custom_css))